# Konvertering av Bergens registeret, Person sammenkoblinger



In [1]:
from hentOgVaskData import hent_og_vask_data
import polars as pl
pl.Config.set_tbl_rows(10)

polars.config.Config

In [2]:
'''
1. Leser inn Data exportert i fra Bergens Access database.
   I fra den tabellen som heter `tblInnmeldte`. Dette gjøres ved å benytte funksjonen `hent_og_vask_data()`. Denne funksjonen er også hardkodet med de som har dobbelt medlemskap mellom registerene. Disse blir satt til "venn" i stedet for medlem av Bergens registeret.
'''

df = hent_og_vask_data()
# Medlems sjekk, det er viktig at dette tallet er nokså likt som rapportert in til statsforvalteren 282
print("Antall medlemmer i Bergens registeret -->", df.filter(pl.col("Kategori").str.starts_with("M")).shape)
print("Alle data i fra excel fil -->", df.shape)

Could not determine dtype for column 7, falling back to string


Antall medlemmer i Bergens registeret --> (279, 46)
Alle data i fra excel fil --> (599, 46)


In [45]:
'''
2. Tar livet av de døde som fortsatt er i live i konverteringen.
   De døde har `Kategori` som er satt til "DØD"
   Kolonnen `DødsDato` er ikke til mye hjelp. Det står noe i `Anmerkning` kolonnen
   Det hele må konverteres manuel etter att de døde er tatt ut.
   De som er umulig å finne ut av får dødsdato "01.01.2025"
'''
df_death = (
    df
        .filter(pl.col("Kategori") == "DØD")
        .select("MedlemID", "FORNAVN", "EFTERNAVN","Anmerkning",
                pl.col("DødsDato"))
)
#df_death.write_excel("etter_konvertering_døde.xlsx")
df_death


MedlemID,FORNAVN,EFTERNAVN,Anmerkning,DødsDato
i64,str,str,str,str
13033,"""Daisy Gunda""","""Akerø""",null,null
12525,"""Herdis""","""Bauge""","""Døde 19.12.2012""",null
14644,"""Kari Berger""","""Bie""",null,null
14338,"""Wenche""","""Dobie""",null,null
11132,"""Alfhild Rachlev""","""Dysthe""",null,null
…,…,…,…,…
14563,"""Egil Kristian""","""Tynæs""",null,null
12748,"""Kirsten""","""Tynæs""",null,null
14690,"""Anni""","""Welzel""","""Meldt død av datteren Anne Des…",null


In [46]:
'''
3. Etterkonvertering av de som skal ha post utsendelse.
   Dette er gjort på tilsvarende måte som i Bergens registeret.
   Det settes en tagg på alle som skal ha post utsendelse.
   Dersom det skal sendes ut på epost, så må en ekstra tagg med epost legges inn
'''

df_utsendelse = pl.concat([
    df
        .filter(pl.col("Utsendelse") == "UB")
        .select("MedlemID", "FORNAVN", "EFTERNAVN")
        .with_columns(pl.lit("Utsendelse").alias("Tag")),

    df
        .filter(
            #(pl.col("Utsendelse") == "UB") &
            (pl.col("Utsendelsesformat") == "E")
        )
        .select("MedlemID", "FORNAVN", "EFTERNAVN")
        .with_columns(pl.lit("Utsendelse Epost").alias("Tag"))
])

df_utsendelse.write_excel("etter_konvertering_utsendelse.xlsx")
#df_utsendelse.filter(pl.col("Tag") == "Utsendelse Epost")
df_utsendelse

MedlemID,FORNAVN,EFTERNAVN,Tag
i64,str,str,str
15535,"""Rita""","""Agdal""","""Utsendelse"""
13033,"""Daisy Gunda""","""Akerø""","""Utsendelse"""
15536,"""Janicke""","""Akerø""","""Utsendelse"""
14800,"""Gry Veronica""","""Alsos""","""Utsendelse"""
14720,"""Fred""","""Amlie""","""Utsendelse"""
…,…,…,…
12747,"""Anne Torill""","""Tutturen""","""Utsendelse Epost"""
13839,"""Signe Elisabet""","""Tynæs""","""Utsendelse Epost"""
13621,"""Marielle""","""Wolfe""","""Utsendelse Epost"""


In [8]:
'''
4. Etterkonvertering av venner
   Dette er de som har "Kategori" == "V"

'''
df_venner = (
    df.filter(pl.col("Kategori") == "V")
    .select("MedlemID", "FORNAVN", "EFTERNAVN")
)

df_venner.write_excel("etter_konvertering_venner.xlsx")
#df_utsendelse.filter(pl.col("Tag") == "Utsendelse Epost")
#df_venner

MedlemID,FORNAVN,EFTERNAVN
i64,str,str
11018,"""Målfrid""","""Aukland"""
11622,"""Gisela""","""Blum-Feldhusen"""
11626,"""Hilde""","""Hjertaas"""
11656,"""Elisabeth Faye""","""Sommerfelt"""
11695,"""Annette Sofie""","""Susaas"""
…,…,…
15607,"""Per Tore""","""Meling"""
15608,"""Sabine""","""Eitel"""
15609,"""Sven""","""Marggraff"""
